In [1]:
# ─── Setup ─────────────────────────────────────────────────────────────────────
!pip install diffusers transformers accelerate --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.8 MB/s eta 0:00:00


In [2]:
import torch
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler
from transformers import BertTokenizer, BertModel
from PIL import Image
import numpy as np
from IPython.display import display
from torchvision import transforms as T

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae").to(device)
vae.eval()

tokenizer    = BertTokenizer.from_pretrained("bert-base-uncased")
text_encoder = BertModel.from_pretrained("bert-base-uncased").to(device)
text_encoder.eval()

unet = UNet2DConditionModel.from_pretrained("apprisco/ufd_pretrain_e11").to(device)
unet.eval()

scheduler = DDPMScheduler.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="scheduler")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [3]:
def generate_images(
    prompts: list[str],
    steps: int = 50,
    guidance_scale: float = 7.5,
    resolution: int = 512,
) -> list[Image.Image]:
    batch_size = len(prompts)
    toks = tokenizer(
        prompts,
        truncation=True,
        padding="max_length",
        max_length=tokenizer.model_max_length,
        return_tensors="pt"
    ).to(device)
    cond_embeds = text_encoder(**toks).last_hidden_state
    uncond_toks = tokenizer(
        [""] * batch_size,
        truncation=True,
        padding="max_length",
        max_length=tokenizer.model_max_length,
        return_tensors="pt"
    ).to(device)
    uncond_embeds = text_encoder(**uncond_toks).last_hidden_state
    encoder_states = torch.cat([uncond_embeds, cond_embeds], dim=0)
    latents = torch.randn(
        (batch_size, unet.config.in_channels, resolution//8, resolution//8),
        device=device
    ) * scheduler.init_noise_sigma
    for t in scheduler.timesteps[:steps]:
        lat_in = torch.cat([latents, latents], dim=0)
        noise_pred = unet(lat_in, t, encoder_hidden_states=encoder_states).sample
        uncond_pred, cond_pred = noise_pred.chunk(2)
        guided = uncond_pred + guidance_scale * (cond_pred - uncond_pred)
        latents = scheduler.step(guided, t, latents).prev_sample
    latents = latents / vae.config.scaling_factor
    with torch.no_grad():
        imgs = vae.decode(latents).sample
    imgs = (imgs / 2 + 0.5).clamp(0, 1)
    out = []
    for i in range(batch_size):
        arr = (imgs[i].cpu().permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        out.append(Image.fromarray(arr))
    return out

In [ ]:
prompts = ["An astronaut on a horse","A futuristic city skyline at sunset", "A red sports car on a mountain road"]
images = generate_images(prompts, steps=50, guidance_scale=7.5)
for img in images:
    display(img)